# SWE Agent Workshop - Autonomous Coding Assistant

## Workshop Flow

1. **Setup Agent** - Use the SWE template & GitHub login
2. **Configure in Notebook** - Add tools & knowledge base  
3. **Background Tasks** - Use Claude Code for async operations
4. **Connect Slack** - Enable team collaboration
5. **Add Notion** - Automate task management

---

## Part 1: Initial Setup

### Step 1: Create Your SWE Agent

1. **Go to template**: https://app.xpander.ai/templates/8a0e7876-8a79-49f4-b6a4-dcdacfb5409b
2. **Login with GitHub** (watch video): https://assets.xpanderai.io/vidoes/workshops/august-2025/01-github-connect.mp4
3. **Test your agent** in the tester tab with a simple prompt
4. **Copy your credentials** from the dashboard for use below

**Important**: Your agent comes with Claude Code tool pre-configured for background tasks!

## Part 2: Notebook Configuration

Now we'll configure your agent locally and add custom capabilities.

In [14]:
# Install dependencies
!pip install "xpander-sdk[agno]" openai requests pypdf pgvector -q
!npm install -g xpander-cli

⠙⠹⠸⠼⠴⠦⠧⠇
changed 189 packages in 836ms
⠇
⠇54 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [ ]:
# Authenticate with Xpander CLI (optional - for advanced features)
!xpander login

In [16]:
import os
from dotenv import load_dotenv

# Load from .env file if exists
load_dotenv()

# Use credentials from your agent created in Step 1
os.environ['XPANDER_ORG_ID'] = os.getenv(
    'XPANDER_ORG_ID', 'your-org-id')  # From dashboard
os.environ['XPANDER_AGENT_ID'] = os.getenv(
    'XPANDER_AGENT_ID', 'your-agent-id')  # From dashboard
os.environ['XPANDER_API_KEY'] = os.getenv(
    'XPANDER_API_KEY', 'your-api-key')  # From dashboard
os.environ['GITHUB_TOKEN'] = os.getenv(
    'GITHUB_TOKEN', 'ghp_xxx')  # Your GitHub personal access token

### Initialize Your SWE Agent

In [17]:
from xpander_sdk import Backend
from agno.agent import Agent

backend = Backend()
swe_agent = Agent(**backend.get_args())
print("✅ SWE Agent initialized!")

✅ SWE Agent initialized!


### Test Multi-Turn Conversations & Session Management

Your agent maintains context across interactions with persistent session management.

In [18]:
# First interaction
response1 = swe_agent.run(
    "What kind of software engineering tasks can you help with?")
print(response1.content)

I can assist with a wide range of software engineering tasks, including but not limited to:

1. **Code Development and Review**: Writing, reviewing, and optimizing code in various programming languages.

2. **Version Control Management**: Handling tasks related to Git, such as creating branches, managing pull requests, and reviewing code changes.

3. **Automated Testing**: Setting up and managing unit tests, integration tests, and other automated testing frameworks.

4. **Continuous Integration/Continuous Deployment (CI/CD)**: Assisting with the setup and maintenance of CI/CD pipelines to automate the software delivery process.

5. **Documentation**: Creating and maintaining technical documentation, including API documentation, user guides, and code comments.

6. **Code Refactoring**: Improving the structure and readability of existing code without changing its functionality.

7. **Bug Fixing and Debugging**: Identifying, diagnosing, and fixing bugs in software applications.

8. **Perf

In [19]:
# Second interaction - it remembers the context
response2 = swe_agent.run(
    "Write a Python function to validate email addresses")
print(response2.content)

To validate email addresses in Python, you can use regular expressions (regex) to define the pattern that a valid email address should match. Here's a Python function that performs this validation:

```python
import re

def is_valid_email(email):
    """
    Validate an email address using a regular expression.

    Args:
        email (str): The email address to validate.

    Returns:
        bool: True if the email address is valid, False otherwise.
    """
    # Define the regex pattern for a valid email address
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    # Use the re.match function to check if the email matches the pattern
    if re.match(email_pattern, email):
        return True
    else:
        return False

# Example usage
email = "example@example.com"
if is_valid_email(email):
    print(f"{email} is a valid email address.")
else:
    print(f"{email} is not a valid email address.")
```

### Explanation:
- **Regex Pattern**: The pattern `^[

In [20]:
# Third interaction - verify session persistence
response3 = swe_agent.run("What did I just ask you to do?")
print(response3.content)

You asked me to write a Python function to validate email addresses.


### Add Custom Tool - GitHub Star

Let's add a local function to star GitHub repositories.

In [21]:
from xpander_sdk import register_tool
import requests


@register_tool
def star_github_repo(repo_full_name: str) -> dict:
    """Star a GitHub repository

    Args:
        repo_full_name: Repository in format 'owner/repo'
    """
    token = os.environ.get("GITHUB_TOKEN")
    if not token:
        return {"error": "No GitHub token configured"}

    response = requests.put(
        f"https://api.github.com/user/starred/{repo_full_name}",
        headers={"Authorization": f"token {token}"}
    )
    return {"success": response.status_code == 204, "repo": repo_full_name}


print("✅ GitHub tool registered!")

✅ GitHub tool registered!


In [22]:
# Reload agent to pick up new tool
swe_agent = Agent(**backend.get_args())
print("Agent reloaded with new tool")

Agent reloaded with new tool


In [23]:
# Test the tool
response = swe_agent.run("Please star the xpander-ai/xpander.ai repository")
print(response.content)

The repository `xpander-ai/xpander.ai` has been successfully starred. If you have any other requests or need further assistance, feel free to let me know!


### Add Knowledge Base - Git Documentation

Load Git best practices documentation for context-aware responses.

In [24]:
from agno.document.chunking.fixed import FixedSizeChunking
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.pgvector import PgVector

# Create knowledge base from GitHub documentation
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://githubtraining.github.io/training-manual/book.pdf"],
    vector_db=PgVector(
        table_name="swe_documentation",
        db_url=swe_agent.storage.db_url,
        auto_upgrade_schema=True,
        schema=swe_agent.storage.schema
    ),
    chunking_strategy=FixedSizeChunking(chunk_size=1000)
)

# Load documents
knowledge_base.load(recreate=False)

# Enable knowledge base for the agent
swe_agent.knowledge = knowledge_base
swe_agent.search_knowledge = True

print("✅ Knowledge base loaded with Git documentation!")

INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Loading knowledge base

INFO Reading: https://githubtraining.github.io/training-manual/book.pdf

INFO Skipped 97 existing/duplicate documents.

INFO Added 0 documents to knowledge base

✅ Knowledge base loaded with Git documentation!


In [25]:
# Test knowledge-enhanced response
response = swe_agent.run("How do I create custom Git aliases for my workflow?")
print(response.content)

Creating custom Git aliases can significantly streamline your workflow by allowing you to execute complex or frequently used commands with simple shortcuts. Here's a step-by-step guide to creating custom Git aliases:

### Step 1: Open Git Configuration
You can add aliases to your Git configuration file. This can be done globally or locally:

- **Globally**: This will apply the alias to all repositories on your system.
  ```bash
  git config --global --edit
  ```

- **Locally**: This will apply the alias only to the current repository.
  ```bash
  git config --edit
  ```

### Step 2: Add Aliases
In the configuration file, you can add aliases under the `[alias]` section. Here are some examples:

- **Shorten Common Commands**:
  ```ini
  [alias]
    co = checkout
    br = branch
    ci = commit
    st = status
  ```

- **Custom Commands**:
  ```ini
  [alias]
    lg = log --oneline --graph --decorate
    amend = commit --amend --no-edit
    unstage = reset HEAD --
  ```

### Step 3: Save a

## Part 3: Background Tasks with Claude Code

Your agent has Claude Code tool pre-configured for running long background tasks. Let's use it to create a PR for the apps-by-agents repository.

### First, check the repository requirements

In [ ]:
# Check what the repository accepts
response = swe_agent.run("""
Check the xpander-ai/apps-by-agents repository on GitHub and tell me:
1. What kind of apps are they accepting?
2. What are the requirements for a PR?
3. What structure should the app follow?
""")
print(response.content)

### Schedule Background Task - Create PR

Now let's task the agent to create a PR with a generated app. Since this uses Claude Code, it will run in the background.

In [29]:
prompt = """
Create a PR to xpander-ai/apps-by-agents repository with a new app. 

Choose one (Or be creative)    
1. **Code Review Assistant** - Streamlit app that reviews code snippets for best practices
2. **Git Commit Message Generator** - Web app that generates conventional commit messages from diffs
3. **API Documentation Generator** - Tool that generates OpenAPI specs from code
4. **SQL Query Builder** - Visual query builder for developers
5. **Regex Helper** - Interactive regex tester and explainer

Use Claude Code to handle this as a background task. Don't wait for completion.
"""

In [30]:
# For example
prompt = """
Create a PR to xpander-ai/apps-by-agents repository with a new app. 
    
**Code Review Assistant** - Streamlit app that reviews code snippets for best practices

Use Claude Code to handle this as a background task. Don't wait for completion.
"""

In [31]:
swe_agent.run(prompt)

RunResponse(content='The task to create a new Streamlit app named **Code Review Assistant** and submit a pull request to the `xpander-ai/apps-by-agents` repository has been initiated. The task is currently queued for execution.\n\nI will monitor the progress and notify you once the task is completed. If you have any other requests or need further assistance, feel free to let me know!', content_type='str', thinking=None, reasoning_content=None, messages=[Message(role='system', content="I am the Software Engineering (SWE) Agent, a specialized AI designed to manage and execute coding-related tasks within an organizational...

In [33]:
# Check if the background task is done
status = swe_agent.run("Did you finish creating the PR?")
print(status.content)

The pull request for the **Code Review Assistant** Streamlit app has been successfully created and submitted to the `xpander-ai/apps-by-agents` repository. Here are the details:

### Key Features Implemented:
- **Multi-language Support**: Supports Python and JavaScript with auto-detection.
- **Best Practice Analysis**: Analyzes naming, structure, and security best practices.
- **Interactive Web UI**: Provides real-time feedback through a user-friendly interface.
- **Security Vulnerability Detection**: Identifies issues like SQL injection, hardcoded secrets, and unsafe `eval` usage.
- **Detailed Issue Reporting**: Offers line numbers and suggestions for improvements.

### Files Created:
- `app.py`: The main Streamlit application with code analysis capabilities.
- `requirements.txt`: Lists the dependencies required (only Streamlit is needed).
- `README.md`: Provides comprehensive documentation with usage examples.

### Pull Request:
- [View Pull Request #107](https://github.com/xpander-a

## Part 4: Connect to Slack

### Build Slack Agent with Zero Infrastructure

1. **Watch setup video**: https://assets.xpanderai.io/vidoes/workshops/august-2025/02-slack.mp4

2. **Create a Slack channel**: `swe-agent-tasks-[yourname]`

3. **Configure Smart Engage**:
   - Go to your agent settings
   - Enable Slack integration
   - Connect to your workspace
   - Select your channel

4. **Test it**: 
   - Watch demo: https://assets.xpanderai.io/vidoes/workshops/august-2025/summarize-pr.mp4
   - Send a message in Slack asking to summarize a PR
   - Your agent will respond!

Your SWE agent now monitors Slack and can execute tasks (including background tasks) from messages!

## Part 5: Add Notion Task Management

### Make Your Agent Fully Autonomous

Create an automated pipeline: **Notion → Slack → Agent → Solution**
1. **Watch setup video**: https://assets.xpanderai.io/Users/david/Desktop/august-workshop/04-notion-template.mp4
2. **Copy Notion template**: https://xpander.notion.site/swe-agent-tasks-template

3. **Add Notion automation**:
   - Trigger: When new issue added to Notion
   - Action: Post task to your Slack channel
   - Result: Agent sees message and starts developing!

### The Complete Flow:
1. Create issue in Notion with description
2. Notion automatically posts to Slack channel
3. Your SWE agent monitors the channel
4. Agent picks up task and starts coding (using Claude Code for long tasks)
5. Solution delivered back to Slack!

**Congratulations! You now have a fully autonomous development pipeline!** 🚀

## Summary

### What You've Built:
✅ **SWE Agent** with Claude Code tool for background tasks  
✅ **Custom tools** like GitHub star functionality  
✅ **Knowledge base** with Git best practices  
✅ **Background task execution** - PR creation without blocking  
✅ **Slack integration** for team collaboration  
✅ **Notion automation** for task management  

### Key Features Demonstrated:
- **Async Operations**: Claude Code handles long-running tasks in background
- **Session Management**: Agent remembers context across interactions
- **Tool Registration**: Add custom capabilities to your agent
- **Knowledge RAG**: Context-aware responses from documentation
- **Zero Infrastructure**: Slack/Notion integration without servers

**Workshop Complete!** 🎉